<a href="https://colab.research.google.com/github/TEE-PLUS/MA/blob/main/Group_Work_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 30 Year Amortization Schedule for a Mortgage

In [2]:

%pip install numpy-financial



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Importing necessary libraries
import pandas as pd
import numpy_financial as npf
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
plt.rcParams["figure.figsize"] = (8,6)
# Defining variables
loan_amount = 1000000
interest_rate = 0.04 / 12
months = 30 * 12
payment = -npf.pmt(interest_rate, months, loan_amount)

# pandas float formatting_
pd.options.display.float_format = '{:,.2f}'.format

# Creating dataframe for amortization schedule
schedule = pd.DataFrame(index=range(1, months+1), columns=['Fixed Payment amount', 'Principal', 'Interest applied', 'Balance'] )
schedule.index.name = "Month number"

# Set initial values for the first row of the dataframe 
schedule.iloc[0]["Fixed Payment amount"] = payment
schedule.iloc[0]["Interest applied"] = (interest_rate * loan_amount)
schedule.iloc[0]["Principal"] = (payment - schedule.iloc[0]["Interest applied"])
schedule.iloc[0]["Balance"] = (loan_amount - schedule.iloc[0]["Principal"])
# Calculate values for the rest of the rows
for i in range(1,months):
    schedule.iloc[i]["Fixed Payment amount"] = payment
    schedule.iloc[i]["Interest applied"] = (schedule.iloc[i-1]["Balance"] * interest_rate)
    schedule.iloc[i]["Principal"] = (payment - schedule.iloc[i]["Interest applied"])
    schedule.iloc[i]["Balance"] = (schedule.iloc[i-1]["Balance"] - schedule.iloc[i]["Principal"])

# Displaying schedule for first 24 months
print(schedule)

             Fixed Payment amount Principal Interest applied    Balance
Month number                                                           
1                        4,774.15  1,440.82         3,333.33 998,559.18
2                        4,774.15  1,445.62         3,328.53 997,113.56
3                        4,774.15  1,450.44         3,323.71 995,663.12
4                        4,774.15  1,455.28         3,318.88 994,207.84
5                        4,774.15  1,460.13         3,314.03 992,747.71
...                           ...       ...              ...        ...
356                      4,774.15  4,695.37            78.78  18,938.53
357                      4,774.15  4,711.02            63.13  14,227.50
358                      4,774.15  4,726.73            47.43   9,500.78
359                      4,774.15  4,742.48            31.67   4,758.29
360                      4,774.15  4,758.29            15.86       0.00

[360 rows x 4 columns]


In [4]:
# Calculating the Total interest paid
Total_interest_paid= schedule['Interest applied'].sum()
print("Total interest paid :", f'${(Total_interest_paid):,.2f}')

Total interest paid : $718,695.06


# 20 Year Amortization Schedule for a Mortgage

In [5]:
# Mortgage detials 
original_balance = 1000000   # Initial Mortgage value
coupon = 0.025                # Annual Interest rate
term = 12 * 20               # Loan Period
monthly_rate = ((1 + coupon)  ** (1/12)) - 1
# payments
periods = range(1, term+1)
interest_payment = npf.ipmt(rate=monthly_rate, per=periods, nper=term, pv=-original_balance)
principal_payment = npf.ppmt(rate=monthly_rate, per=periods, nper=term, pv=-original_balance)

#print("Monthly payment:", round(-npf.pmt(monthly_rate, term, original_balance), 2))

# pandas float formatting_
pd.options.display.float_format = '{:,.2f}'.format

# cash flow table_
cf_data = {'Interest': interest_payment, 'Principal': principal_payment}
cf_table = pd.DataFrame(data=cf_data, index=periods)
cf_table['Payment'] = cf_table['Interest'] + cf_table['Principal']
cf_table['Ending Balance'] = original_balance - cf_table['Principal'].cumsum()
cf_table['Beginning Balance'] = [original_balance] + list(cf_table['Ending Balance'])[:-1]
cf_table = cf_table[['Beginning Balance', 'Payment', 'Interest','Principal', 'Ending Balance']]

print (cf_table)



     Beginning Balance  Payment  Interest  Principal  Ending Balance
1         1,000,000.00 5,285.30  2,059.84   3,225.47      996,774.53
2           996,774.53 5,285.30  2,053.19   3,232.11      993,542.42
3           993,542.42 5,285.30  2,046.53   3,238.77      990,303.65
4           990,303.65 5,285.30  2,039.86   3,245.44      987,058.21
5           987,058.21 5,285.30  2,033.18   3,252.12      983,806.09
..                 ...      ...       ...        ...             ...
236          26,264.00 5,285.30     54.10   5,231.20       21,032.79
237          21,032.79 5,285.30     43.32   5,241.98       15,790.81
238          15,790.81 5,285.30     32.53   5,252.78       10,538.04
239          10,538.04 5,285.30     21.71   5,263.60        5,274.44
240           5,274.44 5,285.30     10.86   5,274.44            0.00

[240 rows x 5 columns]


In [6]:
# Calculating the Total interest paid
print("Total interest:", round(np.sum(cf_table.Interest),2))

Total interest: 268472.79


# 7-1 ARM 30-year mortgage

Total interest: 1472507.45


In [38]:
# Mortgage parameters
original_balance = 1000000   # Initial Mortgage value
term = 12 * 30               # Loan Period
periods = range(1, term+1)
#The New interest(changing every January 1992 - 2015), 8.24 is fixed for 7 years
# interest_rate = [8.24,8.07,7.29,9.22,7.02,7.67,7.03,6.79,8.15,7.07,7.14,5.85,5.87,5.77,6.21,6.18,6.07,5.01,5.09,4.77,3.91,3.34,4.53,3.73]
interest_rate = np.array(pd.read_csv("/content/ARM_interest_rates.csv", header = None).iloc[:, 0])

# payments
beginning_balance = [original_balance]
payment = []
interest_payment = []
principal_payment = []
ending_balance = []
for i in range(term):
    a = -npf.pmt(interest_rate[i]/(12*100), term-i, beginning_balance[i])
    payment.append(a)
    b = (interest_rate[i]/(12*100)) * beginning_balance[i]
    interest_payment.append(b)
    ppl = payment[i] - interest_payment[i]
    principal_payment.append(ppl)
    ebal = beginning_balance[i] - principal_payment[i]
    ending_balance.append(ebal)
    if i < (term-1):
        beginning_balance.append(ending_balance[i])

# cash flow table_
cf_data = {'Interest': interest_payment,
           'Annual Interest Rate': list(interest_rate),
           'Principal': principal_payment, 
          'Payment': payment, 
          'Ending Balance': ending_balance,
          'Beginning Balance': beginning_balance}
cf_table = pd.DataFrame(data=cf_data, index=periods)
cf_table = cf_table[['Beginning Balance', 'Payment', 'Interest','Principal', 'Ending Balance', 'Annual Interest Rate']]

print(cf_table)

     Beginning Balance  Payment  Interest  Principal  Ending Balance  \
1         1,000,000.00 7,505.64  6,866.67     638.97      999,361.03   
2           999,361.03 7,505.64  6,862.28     643.36      998,717.67   
3           998,717.67 7,505.64  6,857.86     647.78      998,069.90   
4           998,069.90 7,505.64  6,853.41     652.22      997,417.67   
5           997,417.67 7,505.64  6,848.93     656.70      996,760.97   
..                 ...      ...       ...        ...             ...   
356          30,426.07 6,142.08     94.57   6,047.50       24,378.57   
357          24,378.57 6,142.08     75.78   6,066.30       18,312.27   
358          18,312.27 6,142.08     56.92   6,085.16       12,227.12   
359          12,227.12 6,142.08     38.01   6,104.07        6,123.04   
360           6,123.04 6,142.08     19.03   6,123.04           -0.00   

     Annual Interest Rate  
1                    8.24  
2                    8.24  
3                    8.24  
4                    8.

In [37]:
# Calculating the Total interest paid
print("Total interest:", round(np.sum(cf_table.Interest),2))

Total interest: 1472507.45
